In [1]:
import json
import numpy as np
import pandas
import string
from pprint import pprint
from datetime import datetime, timedelta
import time
import os
pandas.options.display.max_colwidth = -1
data_dir = '/home/idies/workspace/Temporary/raddick/baseball_scratch/'
print('ok')

ok


In [1]:
print('Reading game log data...')
s = time.time()
gamedata_df = pandas.read_csv('gamedata.csv', low_memory=False, index_col='gameID')

print('Converting data types...')
gamedata_df.loc[:, 'date'] = pandas.to_datetime(gamedata_df['date'], errors='coerce')

print('fixing...')
gamesfixed = []

gamedata_df.loc['PHI194807020', 'length'] = 144
gamesfixed.append('PHI194807020')

print('fixed these games:')
for thisgame in gamesfixed:
    print(thisgame)

print('\n')
e = time.time()

print('Processed {0:,.0f} rows in {1:.0f} seconds.'.format(len(gamedata_df), e-s))


Reading game log data...


NameError: name 'time' is not defined

In [3]:
print('Reading data about games parsed from events info...')
s = time.time()
gameinfo_from_events_df = pandas.DataFrame()
for i in np.arange(1920, 2020, 10):
    filename = data_dir+'gameinfo_from_events_{0:.0f}s.csv'.format(i)
    df = pandas.read_csv(filename, low_memory=False, index_col='gameID')
    gameinfo_from_events_df = pandas.concat((gameinfo_from_events_df,df), sort=False)

print('Converting data types:')
print('date...')
gameinfo_from_events_df.loc[:,'date'] = pandas.to_datetime(gameinfo_from_events_df['date'], errors='coerce')

print('timeofgame...')
gameinfo_from_events_df = gameinfo_from_events_df.assign(length=pandas.to_numeric(np.nan, downcast='integer'))
gameinfo_from_events_df.loc[gameinfo_from_events_df['timeofgame'].notnull(), 'length'] = gameinfo_from_events_df['timeofgame'][gameinfo_from_events_df['timeofgame'].notnull()].apply(lambda x: (int(str(x)[7:9])*60) + (int(str(x)[10:12])))


print('fixing...')
gamesfixed = []
gameinfo_from_events_df.loc['PHA193006231','number'] = 1
gamesfixed.append('PHA193006231')

print('fixed these games:')
for thisgame in gamesfixed:
    print(thisgame)

print('\n')
print('dropping columns that will be overwritten by game log data...')
gameinfo_from_events_df = gameinfo_from_events_df.drop(['date','attendance','hometeam','visteam','site','number','timeofgame','length','wp','lp','save'], axis=1)
gameinfo_from_events_df = gameinfo_from_events_df.drop(['umphome','ump1b','ump2b','ump3b','umplf','umprf'], axis=1)

# Big differences between gw_rbi_batter_id and gwrbi, ignore for now
gameinfo_from_events_df = gameinfo_from_events_df.drop('gwrbi', axis=1)
e = time.time()

print('Processed {0:,.0f} rows in {1:.0f} seconds.'.format(len(gameinfo_from_events_df), e-s))
print('DONE!')


Reading data about games parsed from events info...
Converting data types:
date...
timeofgame...
fixing...
fixed these games:
PHA193006231


dropping columns that will be overwritten by game log data...
Processed 116,966 rows in 4 seconds.
DONE!


In [4]:
print('Identifying columns to add and pair...')
gamedata_columns = gamedata_df.columns.tolist()
columns_to_add = ['starttime','daynight','fieldcond','precip','sky','temp','winddir','windspeed']
columns_to_add += ['usedh','pitches','oscorer','scorer','howscored','howentered']
columns_to_add += ['inputter','inputtime','inputprogvers','translator','edittime', 'comment']
game_from_events_columns = gameinfo_from_events_df.columns.tolist()
#for x in game_from_events_columns:
#    if (x not in gamedata_columns):
        #print(x)
columns_to_pair = [x for x in game_from_events_columns if (x not in gamedata_columns) and (x not in columns_to_add)]

print(columns_to_pair)


print('joining...')
gamedata_df = gamedata_df.join(gameinfo_from_events_df, how='left')
#sorted(gamedata_columns)
print('DONE!')

Identifying columns to add and pair...
[]
DONE!
joining...


In [7]:
gamedata_df.to_csv('gamedata_extended.csv')
df = pandas.read_csv('gamedata_extended.csv', low_memory=False, index_col='gameID')
df.sample(2).T

#os.getcwd()

gameID,PIT190808292,BRO189708281
date,1908-08-29,1897-08-28
gameno,2,1
dayofweek,Sat,Sat
visitor,PHI,LS3
visitor_league,NL,NL
visitor_gameno,112,111
home,PIT,BRO
home_league,NL,NL
home_gameno,117,108
visitor_score,5,11


In [ ]:
#print(sorted(gamedata_columns))
#{name in game data , name in event data}


#gw_rbi_batter_id , gwrbi

#losing_pitcher_id , lp
#saving_pitcher_id , save
#ump_home_id , umphome
#ump_1b_id , ump1b
#ump_2b_id , ump2b
#ump_3b_id , ump3b
#ump_lf_id , umplf
#ump_rf_if , umprf
#winning_pitcher_id , wp


#compare_wp_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['winning_pitcher_id','wp']]
#compare_wp_df = compare_wp_df.dropna()
#compare_wp_df = compare_wp_df.assign(wpdiff = (compare_wp_df['winning_pitcher_id'] == compare_wp_df['wp']))
#compare_wp_df.groupby('wpdiff').size()
#compare_lp_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['losing_pitcher_id','lp']]
#compare_lp_df = compare_lp_df.dropna()
#compare_lp_df = compare_lp_df.assign(lpdiff = (compare_lp_df['losing_pitcher_id'] == compare_lp_df['lp']))
#compare_lp_df.groupby('lpdiff').size()
#compare_sp_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['saving_pitcher_id','save']]
#compare_sp_df = compare_sp_df.dropna()
#compare_sp_df = compare_sp_df.assign(spdiff = (compare_sp_df['saving_pitcher_id'] == compare_sp_df['save']))
#compare_sp_df.groupby('spdiff').size()
#compare_gwrbi_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['gw_rbi_batter_id','gwrbi']]
#compare_gwrbi_df = compare_gwrbi_df.dropna()
#compare_gwrbi_df = compare_gwrbi_df.assign(gwrbidiff = (compare_gwrbi_df['gw_rbi_batter_id'] == compare_gwrbi_df['gwrbi']))
#compare_gwrbi_df[compare_gwrbi_df['gwrbidiff'] == False]

#compare_uh_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_home_id','umphome']]
#compare_uh_df = compare_uh_df.dropna()
#compare_uh_df = compare_uh_df.assign(uhdiff = (compare_uh_df['ump_home_id'] == compare_uh_df['umphome']))
#compare_uh_df.groupby('uhdiff').size()
#compare_u1_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_1b_id','ump1b']]
#compare_u1_df = compare_u1_df.dropna()
#compare_u1_df = compare_u1_df.assign(u1diff = (compare_u1_df['ump_1b_id'] == compare_u1_df['ump1b']))
#compare_u1_df.groupby('u1diff').size()
#compare_u2_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_2b_id','ump2b']]
#compare_u2_df = compare_u2_df.dropna()
#compare_u2_df = compare_u2_df.assign(u2diff = (compare_u2_df['ump_2b_id'] == compare_u2_df['ump2b']))
#compare_u2_df.groupby('u2diff').size()
#compare_u3_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_3b_id','ump3b']]
#compare_u3_df = compare_u3_df.dropna()
#compare_u3_df = compare_u3_df.assign(u3diff = (compare_u3_df['ump_3b_id'] == compare_u3_df['ump3b']))
#compare_u3_df.groupby('u3diff').size()
#compare_ul_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_lf_id','umplf']]
#compare_ul_df = compare_ul_df.dropna()
#compare_ul_df = compare_ul_df.assign(uldiff = (compare_ul_df['ump_lf_id'] == compare_ul_df['umplf']))
#compare_ul_df.groupby('uldiff').size()
#compare_ur_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['ump_rf_id','umprf']]
#compare_ur_df = compare_ur_df.dropna()
#compare_ur_df = compare_ur_df.assign(urdiff = (compare_ur_df['ump_rf_id'] == compare_ur_df['umprf']))
#compare_ur_df.groupby('urdiff').size()


#gamedata_df.loc['ATL198007010']


In [ ]:
#print('Comparing dates...')
#compare_dates_df = gamedata_df.join(gameinfo_from_events_df, how='left', 
#                 lsuffix="_from_game", rsuffix="_from_events_info")[['date_from_game', 'date_from_events_info']]#.sample(2).T
#compare_dates_df = compare_dates_df.dropna()
#compare_dates_df = compare_dates_df.assign(datediff = compare_dates_df['date_from_game'] - compare_dates_df['date_from_events_info'])
##compare_dates_df.groupby('datediff').size()
##datediff
##-4 days    1     
##0 days     116965
## Here's the one discrepancy - it's correct in game data (wrong in events info)
##compare_dates_df[compare_dates_df['datediff'] != timedelta(days=0)]
##gamedata_df.loc['WS1192907250']
#print('Found 1 discrepancy, checked by hand, it is correct in the game data file...')


#print('Comparing attendances...')
#compare_att_df = gamedata_df.join(gameinfo_from_events_df, how='left', 
#                 lsuffix="_from_game", rsuffix="_from_events_info")[['attendance_from_game', 'attendance_from_events_info']]#.sample(2).T
#compare_att_df = compare_att_df.dropna()
#compare_att_df = compare_att_df.assign(att_diff = compare_att_df['attendance_from_game'] - compare_att_df['attendance_from_events_info'])
##compare_att_df.groupby('att_diff').size()
#print('Found {0:,.0f} discrepancies; in all cases, game data is correct.'.format(len(compare_att_df[compare_att_df['att_diff'] != 0])))

#gameinfo_from_events_df['timeofgame'] = pandas.Timedelta(gameinfo_from_events_df['timeofgame'])


#gameinfo_from_events_df['timeofgame'].dropna().apply(lambda x: int(str(x)[10:12]))
#gameinfo_from_events_df['length'].dropna().drop_duplicates().sort_values()
#gamedata_df['length'].loc['PHI194807020']  # game log data: 24 minutes
#gameinfo_from_events_df.loc['PHI194807020']  # events data: 144 minutes

#compare_game_length_df = gamedata_df.join(gameinfo_from_events_df, how='left', lsuffix='_from_gamedata', rsuffix='_from_events')[['length_from_gamedata','length_from_events']].dropna()

#compare_game_length_df = compare_game_length_df.assign(lengthdiff = compare_game_length_df['length_from_gamedata'] - compare_game_length_df['length_from_events'])


#compare_game_length_df['lengthdiff'].dropna().drop_duplicates()

#compare_wp_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['winning_pitcher_id','wp']].sample(2)
#compare_wp_df = compare_wp_df.assign(wpdiff = (compare_wp_df['winning_pitcher_id'] == compare_wp_df['wp']))
#compare_wp_df['wpdiff'].dropna().drop_duplicates()

#compare_lp_df = gamedata_df.join(gameinfo_from_events_df, how='left')[['losing_pitcher_id','lp']].sample(2)
#compare_lp_df = compare_lp_df.assign(lpdiff = (compare_lp_df['losing_pitcher_id'] == compare_lp_df['lp']))
#compare_lp_df['lpdiff'].dropna().drop_duplicates()